## Example Use of David's Finance Module, dfinance
#### See it in action!
This notebook shows an example that implements my custom finance package to backtest an (ineffective) algorithmic trading strategy.

In [1]:
import pandas as pd
import yfinance as yf
import numpy as np

import dfinance as dfin
from dfinance.trading_strategies import SMA_Crossover

%load_ext autoreload
%autoreload 2

In [2]:
# Initialize backtest object
# Initialize portfolio object
my_back = dfin.Backtest()
my_port = dfin.Portfolio()

In [3]:
# Get cash value - can also set this
my_port.cashvalue

100000.0

In [4]:
# # load data from Alpha Vantage csv file
# historical_df = pd.read_csv('gme_minute_data.csv', index_col='time')
# historical_df.index = pd.to_datetime(historical_df.index)
# historical_df.index.name = 'datetime'
# historical_df = historical_df.sort_values(by=['datetime'])

# load data from Yahoo!
data = yf.download(tickers='AAPL', period='60d', progress=False)
data.drop(columns=['Adj Close'])
data.to_csv('aapl_daily_data.csv')
historical_df = pd.read_csv('aapl_daily_data.csv', index_col='Date')

In [5]:
# Process the data from the historical dataframe
my_back.process_historical_data(historical_df,
                                my_port,
                                SMA_Crossover.strategy,
                                sma_short=10,
                                sma_long=15,
                                share_num=10,
                                ticker='AAPL')

2021-11-15 | Short: 149.18 | Long: 148.88 | Short > Long

Buying 10 shares of AAPL for 1494.2999267578125

2021-11-16 | Short: 149.25 | Long: 148.91 | Short > Long
2021-11-17 | Short: 149.37 | Long: 149.07 | Short > Long
2021-11-18 | Short: 149.61 | Long: 149.29 | Short > Long
2021-11-19 | Short: 150.26 | Long: 149.97 | Short > Long
2021-11-22 | Short: 151.34 | Long: 150.85 | Short > Long
2021-11-23 | Short: 152.24 | Long: 151.54 | Short > Long
2021-11-24 | Short: 153.42 | Long: 152.20 | Short > Long
2021-11-26 | Short: 154.29 | Long: 152.58 | Short > Long
2021-11-29 | Short: 155.42 | Long: 153.16 | Short > Long
2021-11-30 | Short: 156.47 | Long: 153.81 | Short > Long
2021-12-01 | Short: 157.99 | Long: 154.78 | Short > Long
2021-12-02 | Short: 158.67 | Long: 155.44 | Short > Long
2021-12-03 | Short: 159.33 | Long: 156.24 | Short > Long
2021-12-06 | Short: 160.11 | Long: 157.36 | Short > Long
2021-12-07 | Short: 160.84 | Long: 158.62 | Short > Long
2021-12-08 | Short: 162.01 | Long: 160

In [6]:
my_port.cashvalue

100266.60018920898

In [7]:
my_port.ledger()

,Ticker,Shares,Share Price,Dollar Amount,Buy/Sell,Transaction Datetime
0,AAPL,10,149.429993,1494.299927,buy,2021-11-15
1,AAPL,-10,178.529999,-1785.299988,sell,2021-12-28
2,AAPL,10,177.259995,1772.599945,buy,2021-12-31
3,AAPL,-10,174.820007,-1748.200073,sell,2022-01-12


In [8]:
my_port.summary_of_transactions()

,Ticker,Shares Held,Current Price,Current Value of Shares


In [9]:
# Buy one share of GME for $100 - what a great price!
my_port.buy_stock('GME', 3, 100, '2021-01-01 00:00:00')

In [10]:
my_port.save_transactions('my_port.csv')

In [11]:
my_port.load_transactions('my_port.csv')
my_port.summary_of_transactions()

,Ticker,Shares Held,Current Price,Current Value of Shares
1,GME,3,102.669998,308.009995
2,Totals,3,N/A,308.009995
